In [1]:
'''pip install numpy==1.23.5
pip install pandas==2.0.3
pip install keras==2.3.0
pip install tensorflow==2.4.3'''
from matplotlib.lines import Line2D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import os
import random
# import ML related libraries
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD, Adam
from keras.models import Model
from keras.layers import Dense, Input, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping, ModelCheckpoint
# import plotting related libraries
from scipy import interpolate
from keras.layers import LeakyReLU
import pickle

In [20]:
def plot_history(history):
    '''
    Plot learning curve using NN training history info
    '''
    hist = pd.DataFrame(history.history)
    hist['epoch'] = history.epoch
    plt.figure(figsize=(6,6))
    plt.xlabel('Epoch',fontsize=4)
    plt.ylabel('Mean Square Error',fontsize=4)
    plt.plot(hist['epoch'], hist['val_mse'],
            label = 'Val Error')
    plt.plot(hist['epoch'], hist['mse'],
            label='Train Error')
    plt.ylim([0,10*10**-3])
    plt.legend()
    plt.show()
def data_import(csv_file_name,input_column_list,output_column_list):
    '''
    used to import dataset and split training and test dataset
    csv_file_name is the dateset root
    reture traing and test datasets
    '''
    raw_data = pd.read_csv(csv_file_name,header=None).dropna()
    x = raw_data.iloc[:,input_column_list].astype(float)
    y = raw_data.iloc[:,output_column_list].astype(float)
    return x, y
def data_melt(x, y,xcolumn,ycolumn,yparameter,yname):
    '''
    Add position as input
    input:
        x: old input 
        y: old output
    output:
        x_new: new input 
        y_new: new output
    '''
    y_label_str = [str(x) for x in ycolumn] # make a list of string of y_label_new
    dataset = pd.concat([x, y],axis=1, ignore_index=True)
    col_names = xcolumn + y_label_str
    dataset.columns = col_names
    dataset = dataset.melt(id_vars=xcolumn, 
        var_name=yparameter, 
        value_name=yname) #for given case, select which HSP to be found
    x_new = dataset.iloc[:, 0:(len(xcolumn)+1)]
    y_new = dataset.iloc[:, (len(xcolumn)+1)]
    print('DATASET')
    print(dataset)
    return x_new, y_new
def MSE(testY, predicY):
    '''
    Get MSE fun
    '''
    MSE=np.sum(np.power((testY - predicY),2))/testY.shape[1]/testY.shape[0]
    return MSE
def get_N_output(Y_train):
    '''
    Get the number of outputs
    '''
    if Y_train.ndim == 1:
        N_outputs = 1
    else:
        N_outputs = Y_train.shape[1]
    return N_outputs
def print_result(Y_test,predict_test_scal,predict_test,scaler_Y,history):
    Y_test_scal = scaler_Y.transform(Y_test.values.reshape(-1,1)).reshape(-1, get_N_output(Y_test))
    MSE_R_scal = MSE(Y_test_scal, predict_test_scal)
    MSE_R = MSE(Y_test.values, predict_test)
    print('MSE_R_scal:')
    print(MSE_R_scal)
    print('MSE_R:')
    print(MSE_R)
    plot_history(history)
    pred_results = []
    sim_results = []
    for i in range(len(predict_test)):
        for j in range(len(predict_test[0])):
            sim_results.append(Y_test.iloc[i, j])
            pred_results.append(predict_test[i][j])
    max_results = max(max(pred_results,sim_results))
    min_results = min(min(pred_results,sim_results))
    fig,ax = plt.subplots(figsize=(5,5))
    ax.plot([min_results, max_results], [min_results, max_results],'b', linewidth=3)
    ax.scatter(pred_results, sim_results,s=10, c='r')
    ax.set_xlabel("Prediction", fontsize=4)
    ax.set_ylabel("FEM testing value", fontsize=4)
    plt.legend()
    plt.show()
def pre_processing(model_type, x, y,train_test_split_test_size,xcolumn,ycolumn,yparameter,yname):
    '''
    Data preprocessing (Uniformly spaced sampling, normalisation, train test split)
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)        
        x: a dataframe of inputs of the whole dataset
        y: a dataframe of outputs of the whole dataset
    outputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        Y_test: Test output
    '''
    # Split dataset
    X_train, X_test, Y_train, Y_test = train_test_split(x,y,test_size=train_test_split_test_size, random_state=3)
    X_train=pd.DataFrame(X_train[:].values)
    X_test =pd.DataFrame(X_test[:].values)
    Y_train=pd.DataFrame(Y_train[:].values)
    Y_test=pd.DataFrame(Y_test[:].values)
    # data reconstruction
    if 'ANN2' in model_type:
        X_train, Y_train = data_melt(X_train, Y_train,xcolumn,ycolumn,yparameter,yname)
        X_test, Y_test_1 = data_melt(X_test, Y_test,xcolumn,ycolumn,yparameter,yname)

    # Normalization
    global scaler_X, scaler_Y
    scaler_X = StandardScaler()
    scaler_Y = StandardScaler()
    scaled_train_X = scaler_X.fit_transform(X_train.to_numpy())
    scaled_test_X = scaler_X.transform(X_test)

    if Y_train.ndim == 1:
      Y_train = np.array(Y_train).reshape(-1,1)#column array
    scaled_train_Y = scaler_Y.fit_transform(Y_train)
    Proc_X_train = scaled_train_X
    Proc_Y_train = scaled_train_Y
    Proc_X_test = scaled_test_X

    return model_type, Proc_X_train, Proc_Y_train, Proc_X_test, Y_test
def seed_tensorflow(seed):
    '''
    Fix ramdom seed
    '''
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['PYTHONHASHSEED'] = str(seed)
def save_model(model_path,ANN):
    '''save the model'''
    ANN.save(model_path)
    print('done saving model')
def build_model_SGD(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,momentum, nesterov,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    callbacks = [
        EarlyStopping(monitor='val_mse', patience=100, verbose=1),
        ModelCheckpoint('best_model.h5', monitor='val_mse', save_best_only=True, verbose=0)
    ]
    model = keras.Sequential([
        keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
        keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
    ])
    model.compile(loss='mse',
                optimizer=keras.optimizers.SGD(learning_rate=learning_rate,momentum=momentum,nesterov=nesterov),
                metrics=['mse'])
    
    history = model.fit(Proc_X_train, Proc_Y_train, batch_size=Batch_size, epochs=Epochs, 
                        verbose=1, validation_split=validation_split, callbacks=callbacks)
    model.summary()
    return model, history
    
def build_model_RMSprop(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,rho,momentum,epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    callbacks = [EarlyStopping(monitor='val_mse', patience=100, verbose=1),ModelCheckpoint('best_model.h5', monitor='val_mse', save_best_only=True, verbose=1)]
    model = keras.Sequential([
        keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
        keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
    ])
    model.compile(loss='mse',
                optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate,rho=rho,momentum=momentum,epsilon=epsilon),
                metrics=['mse'])
    model.summary()
    history = model.fit(Proc_X_train, Proc_Y_train, batch_size=Batch_size, epochs=Epochs, 
                        verbose=1, validation_split=validation_split, callbacks=callbacks)
    return model, history
def build_model_Adagrad(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,initial_accumulator_value, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    callbacks = [EarlyStopping(monitor='val_mse', patience=100, verbose=1),ModelCheckpoint('best_model.h5', monitor='val_mse', save_best_only=True, verbose=1)]
    model = keras.Sequential([
        keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
        keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
    ])
    model.compile(loss='mse',
                  optimizer=keras.optimizers.Adagrad(learning_rate=learning_rate,initial_accumulator_value=initial_accumulator_value,epsilon=epsilon),
                  metrics=['mse'])
    model.summary()
    history = model.fit(Proc_X_train, Proc_Y_train, batch_size=Batch_size, epochs=Epochs, 
                        verbose=1, validation_split=validation_split, callbacks=callbacks)
    return model, history
def build_model_Adadelta(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,rho, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    callbacks = [
        EarlyStopping(monitor='val_mse', patience=100, verbose=1),
        ModelCheckpoint('best_model.h5', monitor='val_mse', save_best_only=True, verbose=1)
    ]
    model = keras.Sequential([
        keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
        keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
    ])
    model.compile(loss='mse',
                  optimizer=keras.optimizers.Adadelta(learning_rate=learning_rate,rho=rho,epsilon=epsilon),
                  metrics=['mse'])
    model.summary()
    history = model.fit(Proc_X_train, Proc_Y_train, batch_size=Batch_size, epochs=Epochs, 
                        verbose=1, validation_split=validation_split, callbacks=callbacks)
    return model, history
def build_model_Adam(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    callbacks = [
        EarlyStopping(monitor='val_mse', patience=100, verbose=1),
        ModelCheckpoint('best_model.h5', monitor='val_mse', save_best_only=True, verbose=0)
    ]
    model = keras.Sequential([
        keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
        keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
    ])
    model.compile(loss='mse',
                  optimizer=keras.optimizers.Adam(learning_rate=learning_rate,beta_1=beta_1,beta_2=beta_2,epsilon=epsilon),
                  metrics=['mse'])
    model.summary()
    history = model.fit(Proc_X_train, Proc_Y_train, batch_size=Batch_size, epochs=Epochs, 
                        verbose=1, validation_split=validation_split, callbacks=callbacks)
    return model, history
def build_model_Adamax(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    callbacks = [
        EarlyStopping(monitor='val_mse', patience=100, verbose=1),
        ModelCheckpoint('best_model.h5', monitor='val_mse', save_best_only=True, verbose=0)
    ]
    model = keras.Sequential([
        keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
        keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
    ])
    model.compile(loss='mse',
                  optimizer=keras.optimizers.Adamax(learning_rate=learning_rate,beta_1=beta_1,beta_2=beta_2,epsilon=epsilon),
                  metrics=['mse'])
    model.summary()
    history = model.fit(Proc_X_train, Proc_Y_train, batch_size=Batch_size, epochs=Epochs, 
                        verbose=1, validation_split=validation_split, callbacks=callbacks)
    return model, history
def build_model_NAdam(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
    callbacks = [
        EarlyStopping(monitor='val_mse', patience=100, verbose=1),
        ModelCheckpoint('best_model.h5', monitor='val_mse', save_best_only=True, verbose=0)
    ]
    model = keras.Sequential([
        keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
        keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
    ])
    model.compile(loss='mse',
                  optimizer=keras.optimizers.Nadam(learning_rate=learning_rate,beta_1=beta_1,beta_2=beta_2,epsilon=epsilon),
                  metrics=['mse'])
    model.summary()
    history = model.fit(Proc_X_train, Proc_Y_train, batch_size=Batch_size, epochs=Epochs, 
                        verbose=1, validation_split=validation_split, callbacks=callbacks)
    return model, history
def get_result_SGD(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test, N_hidden_nodes,learning_rate,momentum, nesterov,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Get the trained model, prediction results, and NN training history
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        N_neurons: Number of Neurons in the hidden layer
        l_rate: Learning rate
    outputs:
        model: Trained model
        predict_test: predicted results
        history: training history
    '''
    #build model
    model, history = build_model_SGD(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,momentum, nesterov,Batch_size, Epochs,BEST_ACT_1,validation_split)
    # Predict
    
    predict_test_scal = model.predict(Proc_X_test)
    if 'ANN2' in model_type:
      predict_test_scal = np.reshape(predict_test_scal, (get_N_output(Y_test), -1)).T
    
    predict_test = scaler_Y.inverse_transform(predict_test_scal)
    return  model, predict_test_scal, predict_test, history


def get_result_RMSprop(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,rho,momentum, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Get the trained model, prediction results, and NN training history
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        N_neurons: Number of Neurons in the hidden layer
        l_rate: Learning rate
    outputs:
        model: Trained model
        predict_test: predicted results
        history: training history
    '''
    #build model
    model, history = build_model_RMSprop(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,rho,momentum, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    # Predict
    
    predict_test_scal = model.predict(Proc_X_test)
    if 'ANN2' in model_type:
      predict_test_scal = np.reshape(predict_test_scal, (get_N_output(Y_test), -1)).T
    
    predict_test = scaler_Y.inverse_transform(predict_test_scal)
    return  model, predict_test_scal, predict_test, history

def get_result_Adagrad(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,initial_accumulator_value, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Get the trained model, prediction results, and NN training history
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        N_neurons: Number of Neurons in the hidden layer
        l_rate: Learning rate
    outputs:
        model: Trained model
        predict_test: predicted results
        history: training history
    '''
    #build model
    model, history = build_model_Adagrad(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,initial_accumulator_value, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    # Predict
    
    predict_test_scal = model.predict(Proc_X_test)
    if 'ANN2' in model_type:
      predict_test_scal = np.reshape(predict_test_scal, (get_N_output(Y_test), -1)).T
    
    predict_test = scaler_Y.inverse_transform(predict_test_scal)
    return  model, predict_test_scal, predict_test, history

def get_result_Adadelta(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,rho, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Get the trained model, prediction results, and NN training history
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        N_neurons: Number of Neurons in the hidden layer
        l_rate: Learning rate
    outputs:
        model: Trained model
        predict_test: predicted results
        history: training history
    '''
    #build model
    model, history = build_model_Adadelta(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,rho, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    # Predict
    
    predict_test_scal = model.predict(Proc_X_test)
    if 'ANN2' in model_type:
      predict_test_scal = np.reshape(predict_test_scal, (get_N_output(Y_test), -1)).T
    
    predict_test = scaler_Y.inverse_transform(predict_test_scal)
    return  model, predict_test_scal, predict_test, history

def get_result_Adam(model_type,Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Get the trained model, prediction results, and NN training history
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        N_neurons: Number of Neurons in the hidden layer
        l_rate: Learning rate
    outputs:
        model: Trained model
        predict_test: predicted results
        history: training history
    '''
    #build model
    model, history = build_model_Adam(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    # Predict
    
    predict_test_scal = model.predict(Proc_X_test)
    if 'ANN2' in model_type:
      predict_test_scal = np.reshape(predict_test_scal, (get_N_output(Y_test), -1)).T
    
    predict_test = scaler_Y.inverse_transform(predict_test_scal)
    return  model, predict_test_scal, predict_test, history

def get_result_Adamax(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Get the trained model, prediction results, and NN training history
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        N_neurons: Number of Neurons in the hidden layer
        l_rate: Learning rate
    outputs:
        model: Trained model
        predict_test: predicted results
        history: training history
    '''
    #build model
    model, history = build_model_Adamax(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    # Predict
    
    predict_test_scal = model.predict(Proc_X_test)
    if 'ANN2' in model_type:
      predict_test_scal = np.reshape(predict_test_scal, (get_N_output(Y_test), -1)).T
    
    predict_test = scaler_Y.inverse_transform(predict_test_scal)
    return  model, predict_test_scal, predict_test, history

def get_result_NAdam(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split):
    '''
    Get the trained model, prediction results, and NN training history
    inputs:
        model_type: 'ANN1' the first architecture (3 inputs and 21 outputs)
                    'ANN2' the second architecture (4 inputs and 1 output)
        Proc_X_train: Processed Training input
        Proc_Y_train: Processed Training output
        Proc_X_test: Processed Test input
        N_neurons: Number of Neurons in the hidden layer
        l_rate: Learning rate
    outputs:
        model: Trained model
        predict_test: predicted results
        history: training history
    '''
    #build model
    model, history = build_model_NAdam(Proc_X_train, Proc_Y_train,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    # Predict
    
    predict_test_scal = model.predict(Proc_X_test)
    if 'ANN2' in model_type:
      predict_test_scal = np.reshape(predict_test_scal, (get_N_output(Y_test), -1)).T
    
    predict_test = scaler_Y.inverse_transform(predict_test_scal)
    return  model, predict_test_scal, predict_test, history
def ActualTraining(model_type,Proc_X_train, Proc_Y_train,Proc_X_test,Y_test,OTHERTUNEDDICT,BEST_OPT,BEST_ACT_1,validation_split):
    print('INFORMATION:')
    print('Used otpimizer:',BEST_OPT)
    print('Used hidden nodes activation function:',BEST_ACT_1)
    D=OTHERTUNEDDICT
    N_hidden_nodes=D['N_hidden_nodes']
    learning_rate=D['learning_rate']
    Batch_size=D['batch_size']
    Epochs=D['nb_epoch']
    print('Used N_hidden_nodes:',N_hidden_nodes)
    print('Used learning_rate:',learning_rate)
    print('Used Batch_size:',Batch_size)
    print('Used Epochs:',Epochs)
    if BEST_OPT=='SGD':
        momentum=D['momentum']
        nesterov=D['nesterov']
        print('Used momentum:',momentum)
        print('Used nesterov:',nesterov)
        ANN, predict_test_scal,predict_test, history = get_result_SGD(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test, N_hidden_nodes,learning_rate,momentum, nesterov,Batch_size, Epochs,BEST_ACT_1,validation_split)
    if BEST_OPT=='RMSprop':
        rho=D['rho']
        momentum=D['momentum']
        epsilon=D['epsilon']
        print('Used rho:',rho)
        print('Used momentum:',momentum)
        print('Used epsilon:',epsilon)
        ANN, predict_test_scal,predict_test, history =get_result_RMSprop(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,rho,momentum, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    if BEST_OPT=='Adagrad':
        initial_accumulator_value=D['initial_accumulator_value']
        epsilon=D['epsilon']
        print('Used initial_accumulator_value:',initial_accumulator_value)
        print('Used epsilon:',epsilon)
        ANN, predict_test_scal,predict_test, history =get_result_Adagrad(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,initial_accumulator_value, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    if BEST_OPT=='Adadelta':
        rho=D['rho']
        epsilon=D['epsilon']
        print('Used rho:',rho)
        print('Used epsilon:',epsilon)
        ANN, predict_test_scal,predict_test, history =get_result_Adadelta(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,rho, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    if BEST_OPT=='Adam':
        beta_1=D['beta_1']
        beta_2=D['beta_2']
        epsilon=D['epsilon']
        print('Used beta_1:',beta_1)
        print('Used beta_2:',beta_2)
        print('Used epsilon:',epsilon)
        ANN, predict_test_scal,predict_test, history =get_result_Adam(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    if BEST_OPT=='Adamax':
        beta_1=D['beta_1']
        beta_2=D['beta_2']
        epsilon=D['epsilon']
        print('Used beta_1:',beta_1)
        print('Used beta_2:',beta_2)
        print('Used epsilon:',epsilon)
        ANN, predict_test_scal,predict_test, history =get_result_Adamax(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    if BEST_OPT=='Nadam':
        beta_1=D['beta_1']
        beta_2=D['beta_2']
        epsilon=D['epsilon']
        print('Used beta_1:',beta_1)
        print('Used beta_2:',beta_2)
        print('Used epsilon:',epsilon)
        ANN, predict_test_scal,predict_test, history =get_result_NAdam(model_type, Proc_X_train, Proc_Y_train, Proc_X_test,Y_test,N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,Batch_size, Epochs,BEST_ACT_1,validation_split)
    return ANN, predict_test_scal,predict_test, history
# Import dataset
def Train_Model(training_dict):
    warnings.filterwarnings('ignore')
    seed_tensorflow(42)
    working_folder=training_dict['working_folder']
    datapathway=training_dict['datapathway']
    input_column_list=training_dict['input_column_list']
    output_column_list=training_dict['output_column_list'] #column of ystart
    yname='Value'      # name of the ultimate output array (HSCP value)
    yparameter='OUTPUT'      #position here but it will be what is the parameter to be tuned (HSCP_name)
    xcolumn=training_dict['xcolumn'] #name of inputs
    ycolumn=training_dict['ycolumn']  #name of outputs (HSCP_name)
    optimizer_search_list=['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
    activation_list1=['relu',LeakyReLU(alpha=0.3),'hard_sigmoid','elu','sigmoid','softmax']
    train_test_split_test_size=training_dict['train_test_split_test_size']
    validation_split=0.1
    model_path=training_dict['model_path']
    x,y=data_import(datapathway,input_column_list,output_column_list)
    # Pre-processing
    global Proc_X_train, Proc_Y_train
    model_type, Proc_X_train, Proc_Y_train, Proc_X_test, Y_test = pre_processing(training_dict['model_type'], x, y,train_test_split_test_size,xcolumn,ycolumn,yparameter,yname)
    
    # Tuning
    def tune_optimizer_ACT(Proc_X_train, Proc_Y_train,optimizer_search_list,activation_list1,optimizer_tuning_number_of_full_update=200,
                       optimizer_tuning_number_sample_used_in_one_training=10):
        model = KerasRegressor(build_fn=create_model, nb_epoch=optimizer_tuning_number_of_full_update, batch_size=optimizer_tuning_number_sample_used_in_one_training, verbose=0) 
        optimizer = optimizer_search_list # 200 hidden nodes
        param_grid = dict(optimizer=optimizer,activation1=activation_list1) 
        grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,verbose=1)
        grid_result = grid.fit(Proc_X_train, Proc_Y_train)
        global BEST_ACT_1,BEST_OPT
        BEST_ACT_1=grid.best_params_['activation1']
        BEST_OPT=grid.best_params_['optimizer']
        print('tuned activation function for hidden layer:')
        print('BEST_ACT_1:',BEST_ACT_1)
        print('tuned optimizer')
        print('BEST_OPT:',BEST_OPT)
    def  create_model(optimizer='adam',activation1='relu'):
        '''
     Build a NN
     inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
     outputs:
        model: Trained model
        history: training history'''
        model = keras.Sequential([
        keras.layers.Dense(200, activation=activation1, input_shape=(Proc_X_train.shape[1],)),
        keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
        ])
        model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mse'])
        model.summary()
        return model
    def get_hype_T_dict(BEST_OPT,BEST_ACT_1,optimizer_search_list):
        SGD_TD= {'momentum':[0,0.2,0.4,0.6,0.8],
                 'nesterov':[True,False]}
        RMSprop_TD={'rho':[0.001,0.01,0.1,0.5,0.9],
                    'momentum':[0,0.2,0.4,0.6,0.8],
                    'epsilon':[1e-9,1e-8,1e-7,1e-6,1e-5]}
        Adagrad_TD={'initial_accumulator_value':[0.001,0.01,0.1,0.5,0.9],
                    'epsilon':[1e-9,1e-8,1e-7,1e-6,1e-5]}
        Adadelta_TD={'rho':[0.001,0.01,0.1,0.5,0.9],
                     'epsilon':[1e-9,1e-8,1e-7,1e-6,1e-5]}
        Adam_TD={'beta_1':[0.8   , 0.8475, 0.895 , 0.9425, 0.99  ],
                 'beta_2':[0.999  ],#set to defult bez of lack of time or the project
                 'epsilon':[1e-9,1e-8,1e-7,1e-6,1e-5]}
        Adamax_TD=Adam_TD
        Nadam_TD=Adam_TD
        TD=[SGD_TD,RMSprop_TD,Adagrad_TD,Adadelta_TD,Adam_TD,Adamax_TD,Nadam_TD]
        hype_search_dict={'learning_rate':[0.001,0.01,0.1,0.2],
                          'N_hidden_nodes':[100,500,1000,1500,2000],
                          'batch_size':[1,8,16,31], 
                          'nb_epoch':[1000,2000,3000],
                          'BEST_ACT_1':[BEST_ACT_1]}
        TD=[SGD_TD,RMSprop_TD,Adagrad_TD,Adadelta_TD,Adam_TD,Adamax_TD,Nadam_TD]
        for i, opt in enumerate(optimizer_search_list):
            if BEST_OPT==opt:
                hype_search_dict.update(TD[i])
                return hype_search_dict
    def create_model_2_SGD(N_hidden_nodes,learning_rate,momentum, nesterov,BEST_ACT_1):
        '''
        Build a NN
        inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
        outputs:
        model: Trained model
        history: training history
        '''
        model = keras.Sequential([
            keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
            keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')])
        model.compile(loss='mse',
                      optimizer=keras.optimizers.SGD(learning_rate=learning_rate,momentum=momentum,nesterov=nesterov),
                      metrics=['mse'])
        print(model.summary())
        return model

    def create_model_2_RMSprop(N_hidden_nodes,learning_rate,rho,momentum, epsilon,BEST_ACT_1):
        '''
        Build a NN
        inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
        outputs:
        model: Trained model
        history: training history
        '''
        model = keras.Sequential([
            keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
            keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
        ])
        model.compile(loss='mse',
                      optimizer=keras.optimizers.RMSprop(learning_rate=learning_rate,rho=rho,momentum=momentum,epsilon=epsilon),
                      metrics=['mse'])
        print(model.summary())
        return model
    
    def create_model_2_Adagrad(N_hidden_nodes,learning_rate,initial_accumulator_value, epsilon,BEST_ACT_1):
        '''
        Build a NN
        inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
        outputs:
        model: Trained model
        history: training history
        '''
        model = keras.Sequential([
            keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
            keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
        ])
        model.compile(loss='mse',
                      optimizer=keras.optimizers.Adagrad(learning_rate=learning_rate,initial_accumulator_value=initial_accumulator_value,epsilon=epsilon),
                      metrics=['mse'])
        print(model.summary())
        return model

    def create_model_2_Adadelta(N_hidden_nodes,learning_rate,rho, epsilon,BEST_ACT_1):
        '''
        Build a NN
        inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
        outputs:
        model: Trained model
        history: training history
        '''
        model = keras.Sequential([
            keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
            keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
        ])
        model.compile(loss='mse',
                      optimizer=keras.optimizers.Adadelta(learning_rate=learning_rate,rho=rho,epsilon=epsilon),
                      metrics=['mse'])
        print(model.summary())
        return model
    def create_model_2_Adam(N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,BEST_ACT_1):
        '''
        Build a NN
        inputs:
            N_hidden_nodes: number of neurons
            input_dim: number of inputs
            N_outputs: number of outputs
            l_rate: learning rate
            Batch_size: Batch size
        outputs:
            model: Trained model
            history: training history
        '''
        model = keras.Sequential([
            keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
            keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
        ])
        model.compile(loss='mse',
                      optimizer=keras.optimizers.Adam(learning_rate=learning_rate,beta_1=beta_1,beta_2=beta_2,epsilon=epsilon),
                      metrics=['mse'])
        print(model.summary())
        return model
    def create_model_2_Adamax(N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,BEST_ACT_1):
        '''
        Build a NN
        inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
        outputs:
        model: Trained model
        history: training history
        '''
        model = keras.Sequential([
           keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
           keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')])
        model.compile(loss='mse',
                      optimizer=keras.optimizers.Adamax(learning_rate=learning_rate,beta_1=beta_1,beta_2=beta_2,epsilon=epsilon),
                      metrics=['mse'])
        print(model.summary())
        return model
    def create_model_2_Nadam(N_hidden_nodes,learning_rate,beta_1,beta_2, epsilon,BEST_ACT_1):
        '''
    Build a NN
    inputs:
        N_hidden_nodes: number of neurons
        input_dim: number of inputs
        N_outputs: number of outputs
        l_rate: learning rate
        Batch_size: Batch size
    outputs:
        model: Trained model
        history: training history
    '''
        model = keras.Sequential([
             keras.layers.Dense(N_hidden_nodes, activation=BEST_ACT_1, input_shape=(Proc_X_train.shape[1],)),
               keras.layers.Dense(get_N_output(Proc_Y_train),activation='linear')
               ])
        model.compile(loss='mse',
                             optimizer=keras.optimizers.Nadam(learning_rate=learning_rate,beta_1=beta_1,beta_2=beta_2,epsilon=epsilon),
                                           metrics=['mse'])
        print(model.summary())
        return model
    def tune_other_para(BEST_OPT,BEST_ACT_1,optimizer_search_list):
        if BEST_OPT=='SGD':
            model = KerasRegressor(build_fn=create_model_2_SGD,verbose=2)
        if BEST_OPT=='RMSprop':
            model = KerasRegressor(build_fn=create_model_2_RMSprop,verbose=2)
        if BEST_OPT=='Adagrad':
            model = KerasRegressor(build_fn=create_model_2_Adagrad,verbose=2)
        if BEST_OPT=='Adadelta':
            model = KerasRegressor(build_fn=create_model_2_Adadelta,verbose=2)
        if BEST_OPT=='Adam':
            model = KerasRegressor(build_fn=create_model_2_Adam,verbose=2)
        if BEST_OPT=='Adamax':
            model = KerasRegressor(build_fn=create_model_2_Adamax,verbose=2)
        if BEST_OPT=='Nadam':
            model = KerasRegressor(build_fn=create_model_2_Nadam,verbose=2)
        param_grid = get_hype_T_dict(BEST_OPT,BEST_ACT_1,optimizer_search_list)
        print('Tuning hyperparameters:')
        print(param_grid)
        grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1,verbose=2)
        grid_result = grid.fit(Proc_X_train, Proc_Y_train)
        global OTHERTUNEDDICT
        OTHERTUNEDDICT=grid.best_params_
        print('tuned hyperparameters:')
        B_N_hidden_nodes=grid.best_params_['N_hidden_nodes']
        B_learning_rate=grid.best_params_['learning_rate']
        B_Batch_size=grid.best_params_['batch_size']
        B_epochs=grid.best_params_['nb_epoch']
        print('B_N_hidden_nodes:',B_N_hidden_nodes)
        print('B_learning_rate:',B_learning_rate)
        print('B_Batch_size:',B_Batch_size)
        print('B_epochs:',B_epochs)
        if BEST_OPT=='SGD':
            B_momentum_SGD=grid.best_params_['momentum']
            B_nesterov_SGD=grid.best_params_['nesterov']
            print('B_momentum_SGD:',B_momentum_SGD)
            print('B_nesterov_SGD:',B_nesterov_SGD)
        if BEST_OPT=='RMSprop':
            B_rho_RMSprop=grid.best_params_['rho']
            B_momentum_RMSprop=grid.best_params_['momentum']
            B_epsilon_RMSprop=grid.best_params_['epsilon']
            print('B_rho_RMSprop:',B_rho_RMSprop)
            print('B_momentum_RMSprop:',B_momentum_RMSprop)
            print('B_epsilon_RMSprop:',B_epsilon_RMSprop)
        if BEST_OPT=='Adagrad':
            B_initial_accumulator_value_Adagrad=grid.best_params_['initial_accumulator_value']
            B_epsilon_Adagrad=grid.best_params_['epsilon']
            print('B_initial_accumulator_value_Adagrad:',B_initial_accumulator_value_Adagrad)
            print('B_epsilon_Adagrad:',B_epsilon_Adagrad)
        if BEST_OPT=='Adadelta':
            B_rho_Adadelta=grid.best_params_['rho']
            B_epsilon_Adadelta=grid.best_params_['epsilon']
            print('B_rho_Adadelta:',B_rho_Adadelta)
            print('B_epsilon_Adadelta:',B_epsilon_Adadelta)
        if BEST_OPT =='Adam':
            B_beta_1_Adam=grid.best_params_['beta_1']
            B_beta_2_Adam=grid.best_params_['beta_2']
            B_epsilon_Adam=grid.best_params_['epsilon']
            print('B_beta_1_Adam:',B_beta_1_Adam)
            print('B_beta_2_Adam:',B_beta_2_Adam)
            print('B_epsilon_Adam:',B_epsilon_Adam)
        if BEST_OPT =='Adamax':
            B_beta_1_Adamax=grid.best_params_['beta_1']
            B_beta_2_Adamax=grid.best_params_['beta_2']
            B_epsilon_Adamax=grid.best_params_['epsilon']
            print('B_beta_1_Adamax:',B_beta_1_Adamax)
            print('B_beta_2_Adamax:',B_beta_2_Adamax)
            print('B_epsilon_Adamax:',B_epsilon_Adamax)
        if BEST_OPT =='Nadam':
            B_beta_1_Nadam=grid.best_params_['beta_1']
            B_beta_2_Nadam=grid.best_params_['beta_2']
            B_epsilon_Nadam=grid.best_params_['epsilon']
            print('B_beta_1_Nadam:',B_beta_1_Nadam)
            print('B_beta_2_Nadam:',B_beta_2_Nadam)
            print('B_epsilon_Nadam:',B_epsilon_Nadam)
    def Save_SC(scaler_X,scaler_Y,working_folder):
        pickle.dump(scaler_X, open(working_folder+r"\scaler_X.pkl",'wb'))
        pickle.dump(scaler_Y, open(working_folder+r"\scaler_Y.pkl",'wb'))
        print('done saving SC')
    tune_optimizer_ACT(Proc_X_train, Proc_Y_train,optimizer_search_list,activation_list1,optimizer_tuning_number_of_full_update=200,
                       optimizer_tuning_number_sample_used_in_one_training=10)
    tune_other_para(BEST_OPT,BEST_ACT_1,optimizer_search_list)
    ANN, predict_test_scal,predict_test, history=ActualTraining(model_type,Proc_X_train, Proc_Y_train,Proc_X_test,Y_test,OTHERTUNEDDICT=OTHERTUNEDDICT,BEST_OPT=BEST_OPT,BEST_ACT_1=BEST_ACT_1,validation_split=validation_split)
    print_result(Y_test=Y_test,predict_test_scal=predict_test_scal,predict_test=predict_test,scaler_Y=scaler_Y, history=history)        
    save_model(model_path=model_path,ANN=ANN)
    Save_SC(scaler_X,scaler_Y,working_folder)
    return ANN, scaler_X,scaler_Y

In [21]:
def Use_Model(ANN_loc,SX_loc,SY_loc,input_list):
    model=keras.models.load_model(ANN_loc)
    SX=pickle.load(open(SX_loc,'rb'))
    SY=pickle.load(open(SY_loc,'rb'))
    input_S=SX.transform(np.array([input_list]))
    answer_S=model.predict(input_S)
    answer=SY.inverse_transform(answer_S)
    print(answer)

In [22]:
training_dict= {'working_folder':r"C:\Users\wingt\OneDrive\Desktop\Individual Project ML\1b_fusion_zone\1b_x_fb",
                'datapathway':r"C:\Users\wingt\OneDrive\Desktop\Individual Project ML\1b_fusion_zone\1b_x_fb\dataset_1b(Sheet1).csv",
                'input_column_list':[1,2,3,4,5,6],
                'output_column_list':[7],
                'xcolumn':['heat', 'speed', 'ra','rl','rv','y_fb'],
                'ycolumn':['x_fb'],
                'train_test_split_test_size':0.2,
                'model_type':'ANN1'}
model_path=r"C:\Users\wingt\OneDrive\Desktop\Individual Project ML\1b_fusion_zone\1b_x_fb\saved_model"
training_dict.update({'model_path':model_path})

In [ ]:
ANN_1a, scaler_X_1a,scaler_Y_1a=Train_Model(training_dict=training_dict)

Fitting 5 folds for each of 42 candidates, totalling 210 fits
Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 200)               1400      
_________________________________________________________________
dense_17 (Dense)             (None, 1)                 201       
Total params: 1,601
Trainable params: 1,601
Non-trainable params: 0
_________________________________________________________________
tuned activation function for hidden layer:
BEST_ACT_1: relu
tuned optimizer
BEST_OPT: Adam
Tuning hyperparameters:
{'learning_rate': [0.001, 0.01, 0.1, 0.2], 'N_hidden_nodes': [100, 500, 1000, 1500, 2000], 'batch_size': [1, 8, 16, 31], 'nb_epoch': [1000, 2000, 3000], 'BEST_ACT_1': ['relu'], 'beta_1': [0.8, 0.8475, 0.895, 0.9425, 0.99], 'beta_2': [0.999], 'epsilon': [1e-09, 1e-08, 1e-07, 1e-06, 1e-05]}
Fitting 5 folds for each of 6000 candidates, t

In [8]:
ANN_loc=r"C:\Users\wingt\OneDrive\Desktop\Individual Project ML\1a_near_field\saved_model"
SX_loc=r"C:\Users\wingt\OneDrive\Desktop\Individual Project ML\1a_near_field\scaler_X.pkl"
SY_loc=r"C:\Users\wingt\OneDrive\Desktop\Individual Project ML\1a_near_field\scaler_Y.pkl"
input_list=[1920,1.4,31,21,18]
Use_Model(ANN_loc,SX_loc,SY_loc,input_list)

[[408.23285]]


In [ ]:
1920,1.4,31,21,18,408.57